# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Требования к выполнению проекта

1. Загрузите и подготовьте данные. Поясните порядок действий.
2. Обучите и проверьте модель для каждого региона:
2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
2.2. Обучите модель и сделайте предсказания на валидационной выборке.
2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
2.5. Проанализируйте результаты.
3. Подготовьтесь к расчёту прибыли:
3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
3.3. Напишите выводы по этапу подготовки расчёта прибыли.
4. Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
4.1. Выберите скважины с максимальными значениями предсказаний.
4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
4.3. Рассчитайте прибыль для полученного объёма сырья.
Посчитайте риски и прибыль для каждого региона:
5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from scipy import stats as st
import warnings
warnings.filterwarnings('ignore')

In [2]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
df2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [6]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
print('Количество дубликатов в df0:', df0.duplicated().sum())
print('Количество дубликатов в df1:', df1.duplicated().sum())
print('Количество дубликатов в df2:', df2.duplicated().sum())

Количество дубликатов в df0: 0
Количество дубликатов в df1: 0
Количество дубликатов в df2: 0


In [10]:
print('Информация о df0:', df0.describe())
print('------------------------------------------------------------------')
print('Информация о df1:', df1.describe())
print('------------------------------------------------------------------')
print('Информация о df2:', df2.describe())

Информация о df0:                   f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
------------------------------------------------------------------
Информация о df1:                   f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598    

In [11]:
display(df0.nunique())
print('-------------')
display(df1.nunique())
print('-------------')
display(df2.nunique())

id          99990
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64

-------------


id          99996
f0         100000
f1         100000
f2         100000
product        12
dtype: int64

-------------


id          99996
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64

In [12]:
display(df0.corr())
print('-------------')
display(df1.corr())
print('-------------')
display(df2.corr())

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


-------------


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


-------------


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


# Описание данных

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

# Условие задачи

1. Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
2. При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
3. Бюджет на разработку скважин в регионе — 10 млрд рублей.
4. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
5. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**Выводы:**
1. Все датасеты полные, без пропусков и подготовлены к исследованию, все типы данных указаны правильно, дубликатов нет. 
2. Имеются повторяющиеся значения в столбцах id. Скорее всего данные из одной и той же скважины брались несколько раз.
3. У датасета с индексом 1 уникальных значений в product 12. Интересно как поведет себя модель.
4. Задача 1 пункта проекта выполнена, можно переходить к обучению и проверке модели.
5. Столбец id для обучения модели не пригодится.

## Обучение и проверка модели

In [13]:
df0 = df0.drop(['id'], axis=1)
df1 = df1.drop(['id'], axis=1)
df2 = df2.drop(['id'], axis=1)

In [14]:
train_0, valid_0 = train_test_split(df0, test_size=0.25, random_state=12345)
train_1, valid_1 = train_test_split(df1, test_size=0.25, random_state=12345)
train_2, valid_2 = train_test_split(df2, test_size=0.25, random_state=12345)

*Пояснение:*
Разделил датафреймы на обучающую и валидационную выбороки в пропорциях 3:1 в соответствии с заданием.

In [15]:
print(train_0.shape, valid_0.shape)
print(train_1.shape, valid_1.shape)
print(train_2.shape, valid_2.shape)

(75000, 4) (25000, 4)
(75000, 4) (25000, 4)
(75000, 4) (25000, 4)


In [16]:
def features_target_split(data):
    features = data.drop(['product'], axis=1)
    target = data['product']
    return features, target

train_features_0, train_target_0 = features_target_split(train_0)
train_features_1, train_target_1 = features_target_split(train_1)
train_features_2, train_target_2 = features_target_split(train_2)

valid_features_0, valid_target_0 = features_target_split(valid_0)
valid_features_1, valid_target_1 = features_target_split(valid_1)
valid_features_2, valid_target_2 = features_target_split(valid_2)

*Пояснение:*
Разделил выборки на признак и целевой признак в функции features_target_split()

In [17]:
def df_to_standardscaler(data):
    scaler = StandardScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    return data

train_features_0 = df_to_standardscaler(train_features_0)
train_features_1 = df_to_standardscaler(train_features_1)
train_features_2 = df_to_standardscaler(train_features_2)

valid_features_0 = df_to_standardscaler(valid_features_0)
valid_features_1 = df_to_standardscaler(valid_features_1)
valid_features_2 = df_to_standardscaler(valid_features_2)

*Пояснение:*
Обязательно использую стандартизацию данных, на которой погарел в прошлом проекте. Теперь ни за что не забуду))) Использую StandardScaler, преобразовав данные функцией transform(). Для упрощения работы создал функцию data_to_standardscaler.

In [18]:
def rmse_model(train_features, train_target, valid_features, valid_target):
    model = LinearRegression().fit(train_features, train_target)
    predicted = model.predict(valid_features)
    mse_0 = mean_squared_error(valid_target, predicted)
    rmse_0 = mse_0 ** 0.5
    print('RMSE модели {:.2f}'.format(rmse_0))
    return predicted

*Пояснение:*
В соответсвии с заданием использовал модель "Линейная регрессия". Соответственно для расчёта RMSE написал одноименную функцию.

In [19]:
print('Регион 0')
print('Cредний запас сырья в регионе {:.2f} тыс. баррелей на 1 скважину'.format(df0['product'].mean()))
predicted_valid_0  = rmse_model(train_features_0, train_target_0, valid_features_0, valid_target_0)
print('Cредняя прибыль с добытого сырья {:.2f} млрд. руб.'.format(df0['product'].mean()*450000*200/1e9))

Регион 0
Cредний запас сырья в регионе 92.50 тыс. баррелей на 1 скважину
RMSE модели 37.58
Cредняя прибыль с добытого сырья 8.33 млрд. руб.


In [20]:
print('Регион 1')
print('Cредний запас сырья в регионе {:.2f} тыс. баррелей на 1 скважину'.format(df1['product'].mean()))
predicted_valid_1  = rmse_model(train_features_1, train_target_1, valid_features_1, valid_target_1)
print('Cредняя прибыль с добытого сырья {:.2f} млрд. руб.'.format(df1['product'].mean()*450000*200/1e9))

Регион 1
Cредний запас сырья в регионе 68.83 тыс. баррелей на 1 скважину
RMSE модели 0.91
Cредняя прибыль с добытого сырья 6.19 млрд. руб.


In [21]:
print('Регион 2')
print('Cредний запас сырья в регионе {:.2f} тыс. баррелей на 1 скважину'.format(df2['product'].mean()))
predicted_valid_2  = rmse_model(train_features_2, train_target_2, valid_features_2, valid_target_2)
print('Cредняя прибыль с добытого сырья {:.2f} млрд. руб.'.format(df2['product'].mean()*450000*200/1e9))

Регион 2
Cредний запас сырья в регионе 95.00 тыс. баррелей на 1 скважину
RMSE модели 40.03
Cредняя прибыль с добытого сырья 8.55 млрд. руб.


**Вывод:**
1. Написал очень много функций для обучения и проверок модели. 
2. Обученная модель методом "Линейная регрессия" показала, что в лучшая RMSE модель в 1 регионе, но как по мне, то этому доверять не стоит, вспоминая, что уникальных значений в product всего 12. Соответственно в регионе 1 средний запас сырья - почти 69 тыс.баррелей, а в 0 и 2 - 92,5 и 95 тыс.баррелей соответственно, что значительно повлияет на прибыль.
3. Модель хорошо обучилась и лучше предсказывает в регионе 1.
4. Для определения необходимой скважены обязательно нужно будет подсчитать прибыль и риски.

## Подготовка к расчёту прибыли

In [22]:
valid_target_0.reset_index(drop=True, inplace=True)
valid_target_1.reset_index(drop=True, inplace=True)
valid_target_2.reset_index(drop=True, inplace=True)

predicted_valid_0 = pd.Series(predicted_valid_0)
predicted_valid_1 = pd.Series(predicted_valid_1)
predicted_valid_2 = pd.Series(predicted_valid_2)

*Посянение:* Ключевые значения сделал пригодными для работы.

In [23]:
min_v = 50000000 / 450 / 1000

print('Минимальный средний объём сырья {:.3f} тыс.баррелей'.format(min_v))

Минимальный средний объём сырья 111.111 тыс.баррелей


In [24]:
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450 * 1000 * selected.sum() / 1e9


#def revenue(target, probabilities, count):
#    probs_sorted = probabilities.sort_values(ascending=False)
#    selected = target[probs_sorted.index][:count]
#    return (profit_from_one_product * selected.sum())

**Вывод:**
1. Рассчитал минимальный средний объём сырья в месторождениях региона. Считал по формуле:
Цена за бурение 1 скважины = бюджет / 200 скважин, соответственно минимальный средний объём сырья в месторождениях региона = Цена за бурение 1 скважины / Доход с каждой единицы продукта

2. Получилось, что расходы по добыче в каждой скважине региона должно быть в среднем 111.11 тыс.баррелей нефти.

3. Написал функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели distribution_revenue, принимающая на вход предсказания, истинные значения целевого признака и количество скважин и считающая общую прибыль в млрд.руб в регионе.
4. В общем и целом подготовил данные для расчета по технике Bootstrap и статистического анализа.

## Расчёт прибыли и рисков 

In [25]:
state = np.random.RandomState(12345)
def distribution_of_revenue(target, probabilities):
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample, 200))

    values = pd.Series(values)
    mean = values.mean() - 10 # вычитаем расходы в 10 млрд. руб. на разработку скважин
    print('Средняя прибыль {} млрд.руб.'.format(mean))
    
    lower = np.quantile(values, 0.025) - 10
    upper = np.quantile(values, 0.975) - 10
    print('95%-й доверительный интервал  {} - {} млрд.руб.'.format(lower, upper)) # Отрицательный - убыток, положительный - прибыль
    
    loss_risk = st.percentileofscore(values, 10)
    print('Риск убытков {} %'.format(loss_risk))
    return 

In [26]:
distribution_of_revenue(valid_target_0, predicted_valid_0)

Средняя прибыль 0.42651131389323993 млрд.руб.
95%-й доверительный интервал  -0.1044714661254158 - 0.9479763533583689 млрд.руб.
Риск убытков 6.1 %


In [27]:
distribution_of_revenue(valid_target_1, predicted_valid_1)

Средняя прибыль 0.5182580631011628 млрд.руб.
95%-й доверительный интервал  0.12812323143308468 - 0.9536129820669075 млрд.руб.
Риск убытков 0.3 %


In [28]:
distribution_of_revenue(valid_target_2, predicted_valid_2)

Средняя прибыль 0.42022287822227433 млрд.руб.
95%-й доверительный интервал  -0.11585260916001161 - 0.9896299398445745 млрд.руб.
Риск убытков 6.2 %


**Вывод:**
1. Применил технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
2. Нашел среднюю прибыль, 95%-й доверительный интервал и риск убытков.
3. Написал функцию distribution_of_revenue с техникой Bootstrap с 1000 повторениями для 500 возможных скважин, 200 из которых будут отобраны для бурения. Функция считает среднюю прибыль, 95%-й доверительный интервал и риск убытков по целевому признаку и предсказаниям.
4. При Bootstrap с 1000 выборок, распределение средних находится в районе 10 млрд. Для регионов 0 и 2 в районе 10,4 млрд. для региона 1 в 105 млрд .  -->
5. 95% доверительный интервал только для региона 2 выше 10,5 млрд.рублей.
6. Риски получения убытков наименьшие (менее 1%) только во 1 регионе. (df1) (geo_data_1)
7. Наиболее предпочтительным регионом для бурения скважин по средней приблыли и минимальному риску убытков - **Регион 1.** (df1) (geo_data_1)

# Вывод проекта

1. Обучение модели методом "Линейная регрессия" в 3 регионах показала, что в регионе 1 лучшая RMSE, модель предсказывает наличие запасов нефти с точность до 0.9 тыс.баррелей, в то время как в регионе 0 и 2 достигается разброс около 40 тыс.баррелей.
2. В общем и целом все регионы подходят по среднему количеству баррелей в скважинах.Однако, на случайной выборке из 1000 повторений модель предсказывает среднюю прибыль и 95%-й доверительный интервал операционного дохода лучше в Регионе 1.
3. Чтобы получить максимальную прибыль при минимальных рисках необходимо использовать Регион 1 для добычи сырья, т.к.  это лучший регион исходя из заданных данных.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован